<img src="../../data/images/gempy_logo.png" />

# <center> From Maps to Models - Tutorials for structural geological modeling using GemPy and GemGIS</center>

# Model 3 - Faulted Layers


This third notebook in this tutorial series builds upon the [first notebook](model1_Horizontal_Layers.ipynb) ([notebook on Github](https://nbviewer.org/github/cgre-aachen/gemgis_data/blob/main/notebooks/01_basic_modeling/model1_Horizontal_Layers.ipynb)) and [second notebook](model2_Folded_Layers.ipynb) ([notebook on Github](https://nbviewer.org/github/cgre-aachen/gemgis_data/blob/main/notebooks/01_basic_modeling/model2_Folded_Layers.ipynb)) where horizontal and folded layers were modeled, respectively. This notebook illustrates how to create a simple sample model of faulted layers in `GemPy`. The model consists of three parallel layers and two faults and has an extent of 1000 m by 1000 m with a vertical extent of 600 m. 

If you have not gone through the introduction notebook for the course, please check it out: [Introduction Notebook](../00_introduction_to_structural_modeling.ipynb) ([notebook on Github](https://nbviewer.org/github/cgre-aachen/gemgis_data/blob/main/notebooks/00_introduction_to_structural_modeling.ipynb))



<div class="alert alert-block alert-success">
<b>In this tutorial, you will learn the following:</b> <br>
- Get an understanding of how faults are modeled/displayed in GemPy<br>
- How to build a simple model consisting of <b>faulted layers</b> belonging to <b>one Series</b><br>

</div>

## Contents

1. [Installing GemPy](#installing-gempy)
2. [Importing Libraries](#importing-libraries)
3. [Data Preparation](#data-preparation)
    1. [Importing Interface Points](#importing-interface-points)
    2. [Importing Orientations](#importing-orientations)
4. [GemPy Model Calculation](#gempy-model-calculation)
    1. [Creating the GemPy Model](#creating-the-gempy-model)
    2. [Data Initiation](#data-initiation)
    3. [Inspecting the Surfaces](#inspecting-the-surfaces)
    4. [Inspecting the Input Data](#inspecting-the-input-data)
    5. [Map Stack to Surfaces](#map-stack-to-surfaces)
    6. [Setting Faults](#setting-faults)
    7. [Plotting Input Data in 2D](#plotting-the-input-data-in-2d)
    8. [Plotting Input Data in 3D](#plotting-the-input-data-in-3d)
    9. [Setting the Interpolator](#setting-the-interpolator)
    10. [Computing the Model](#computing-the-model)
5. [Model Visualization and Post-Processing](#model-visualization-and-post-processing)
    1. [Visualizing Cross Sections of the Computed Model](#visualizing-cross-sections-of-the-computed-model)
    2. [Visualizing the computed model in 3D](#visualizing-the-computed-model-in-3d)
6. [Conclusions](#conclusions)
7. [Outlook](#outlook)
8. [Licensing](#licensing)



<img src="../images/model3.png" width=500/>


In reality, most geological settings are formed by a concatenation of depositional phases partitioned by unconformity boundaries and subjected to tectonic stresses that displace and deform the layers. While the interpolation is able to represent realistic folding – given enough data – the method fails to describe discontinuities. To overcome this limitation, it is possible to combine several scalar fields to recreate the desired result. So far, the implemented discontinuities in GemPy are unconformities and infinite faults. Both types are computed by specific combinations of independent scalar fields. To display the offset of faults, a drift function is calculated. 


Faults require the same input data as layers, interface points and orientations. `GemPy` has to be told that faults are present in the model in order to treat them as such. This is done with the function `geo_model.set_is_fault()` where you pass the name of the faults. 


<img src="../../data/images/model3_fig1.PNG" width=500/>
Source: <a href="https://doi.org/10.5194/gmd-12-1-2019">de la Varga et al. (2019)</a>

<a id='installing-gempy'></a>

# Installing GemPy

If you have not installed `GemPy` yet, please follow the [installation instructions](https://docs.gempy.org/installation.html). If you encounter any issues, feel free to open a new discussion at [GemPy Discussions](https://github.com/cgre-aachen/gempy/discussions). If you encounter an error in the installation process, feel free to also open an issue at [GemPy Issues](https://github.com/cgre-aachen/gempy/issues). There, the `GemPy` development team will help you out. 

<a id='importing-libraries'></a>

# Importing Libraries

For this notebook, we need the `pandas` library for the data preparation and `matplotlib` for plotting and of course the `gempy` library. Any warnings that may appear can be ignored for now. 

In [ ]:
import pandas as pd
import gempy as gp
import matplotlib.pyplot as plt

<a id='data-preparation'></a>
# Data Preparation

For this model, the only thing that needs to be done is loading the already created interface points and orientations. In the next tutorials, you will create the data yourself and process it further to make it usable for GemPy. 

<a id='importing-interface-points'></a>
## Importing Interface Points

We are using the `pandas` library to load the interface points that were prepared beforehand and stored as CSV-file. The only information that is needed are the location of the interface point (`X`, `Y`, `Z`) and the `formation` it belongs to. You may have to adjust the `delimiter` when loading the file.

In [ ]:
interfaces = pd.read_csv('../data/model3/model3_interfaces.csv', 
                         delimiter = ';')
interfaces.head()

<a id='importing-orientations'></a>

## Importing Orientations

The orientations will also be loaded using `pandas`. In addition to the location and the formation the orientation belongs to, the dip value, azimuth value (dip direction) and a polarity value (mostly set to 1 by default) needs to be provided. As the model will feature faulted layers, the dip and the azimuth values may be variable. Orientations are provided for all three modeled layers. 

In [ ]:
orientations = pd.read_csv('../data/model3/model3_orientations.csv', 
                           delimiter=';')
orientations.head()

<a id='gempy-model-calculation'></a>

# GemPy Model Calculation

The following part presents the main steps of creating a model in `GemPy`. 

The creation of a `GemPy` Model follows particular steps which will be performed in the following:

1. Create new model: `gp.create_model()`
2. Data Initiation: `gp.init_data()`
3. Map Stack to Surfaces: `gp.map_stack_to_surfaces()`
4. Setting Faults: `geo_model.set_is_fault()`
5. Set the Interpolator: `gp.set_interpolator()`
6. Computing the Model: `gp.compute_model()`

<a id='creating-the-gempy-model'></a>

## Creating the GemPy Model

The first step is to create a new empty `GemPy` model by providing a name for it. 

In [ ]:
geo_model = gp.create_model('Model3_Faulted_Layers')
geo_model

<a id='data-initiation'></a>

## Data Initiation

During this step, the `extent` of the model (`xmin`, `xmax`, `ymin`, `ymax`, `zmin`, `zmax`) and the `resolution` in `X`, `Y`and `Z` direction (`res_x`, `res_y`, `res_z`, equal to the number of cells in each direction) will be set using lists of values. 

The interface points (`surface_points_df`) and orientations (`orientations_df`) will be passed as `pandas` `DataFrames`. 

In [ ]:
gp.init_data(geo_model=geo_model, 
             extent=[0, 1000, 0, 1000, -600, 0], 
             resolution=[100, 100, 100],
             surface_points_df=interfaces,
             orientations_df=orientations,
             default_values=True)

<a id='inspecting-the-surfaces'></a>

## Inspecting the Surfaces

The model consists of three different layers or surfaces now which all belong to the `Default series`. During the next step, the proper `Series` will be assigned to the surfaces. Using the `surfaces`-attribute again, we can check which layers were loaded.

In [ ]:
geo_model.surfaces

<a id='inspecting-the-input-data'></a>

## Inspecting the Input Data

The loaded interface points and orientations can again be inspected using the `surface_points`- and `orientations`-attributes. Using the `df`-attribute of this object will convert the displayed table in a `pandas` `DataFrame`.

In [ ]:
geo_model.surface_points.df.head()

In [ ]:
geo_model.orientations.head()

<a id='map-stack-to-surfaces'></a>

## Map Stack to Surfaces

During this step, all three layers of the model are assigned to the `Strata1` series. We know that the layers modeled here are parallel. If the layers were not parallel as shown in the next models, multiple series would be defined. Further, we define two separate series for `Fault1` and `Fault2`. We will also add a `Basement` here (`geo_model.add_surfaces('Basement')`). The order within one series also defines the age relations within this series and has to be according to the depositional events of the layers.

In [ ]:
gp.map_stack_to_surfaces(geo_model,
                         {
                          'Fault1': ('Fault1'),
                          'Fault2': ('Fault2'),
                          'Strata1': ('Layer1', 'Layer2', 'Layer3'),                          
                         },
                         remove_unused_series=True)
geo_model.add_surfaces('Basement')
geo_model.surfaces

In [ ]:
geo_model.stack

<a id='setting-faults'></a>

## Setting Faults 

In addition to `Series` for the layers, we defined two `Series` containing the two faults. In order for `GemPy` to recognize the faults as faults, we need to define them as faults (`geo_model.set_is_fault()`). 

In [ ]:
geo_model.set_is_fault(['Fault1', 'Fault2'])

<a id='plotting-the-input-data-in-2d'></a>

## Plotting the input data in 2D using Matplotlib

The input data can now be visualized in 2D using `matplotlib`. This might for example be useful to check if all points and measurements are defined the way we want them to. Using the function `plot_2d()`, we attain a 2D projection of our data points onto a plane of chosen direction (we can choose this attribute to be either `'x'`, `'y'`, or `'z'`).

In [ ]:
gp.plot_2d(geo_model, 
           direction='z', 
           show_lith=False, 
           show_boundaries=False)
plt.grid()

<a id='plotting-the-input-data-in-3d'></a>

## Plotting the input data in 3D using PyVista

The input data can also be viszualized using the `pyvista` package. In this view, the interface points are visible as well as the orientations (marked as arrows) which indicate the normals of each orientation value. 

The `pyvista` package requires the Visualization Toolkit (VTK) to be installed.

In [ ]:
gp.plot_3d(geo_model, 
           image=False, 
           plotter_type='basic', 
           notebook=True)

<a id='setting-the-interpolator'></a>
## Setting the interpolator

Once we have made sure that we have defined all our primary information, we can continue with the next step towards creating our geological model: preparing the input data for interpolation.

Setting the interpolator is necessary before computing the actual model. Here, the most important kriging parameters can be defined. 

In [ ]:
gp.set_interpolator(geo_model,
                    compile_theano=True,
                    theano_optimizer='fast_compile',
                    verbose=[],
                    update_kriging=False
                    )

<a id='computing-the-model'></a>

## Computing the model

At this point, we have all we need to compute our full model via `gp.compute_model()`. By default, this will return two separate solutions in the form of arrays. The first provides information on the lithological formations, the second on the fault network in the model, which is not present in this example. 

In [ ]:
sol = gp.compute_model(geo_model, 
                       compute_mesh=True)

In [ ]:
sol

In [ ]:
geo_model.solutions

<a id='model-visualization-and-post-processing'></a>

# Model Visualization and Post-Processing

<a id='visualizing-cross-sections-of-the-computed-model'></a>

## Visulazing Cross Sections of the computed model

Cross sections in different `direction`s and at different `cell_number`s can be displayed. Here, we see the horizontal layers in the one direction and the faulted layers in the other direction.

In [ ]:
gp.plot_2d(geo_model, 
           direction=['x', 'x', 'y', 'y'], 
           cell_number=[25, 50, 25, 75], 
           show_topography=False, 
           show_data=True)

Next to the lithology data, we can also plot the calculated scalar field.

In [ ]:
gp.plot_2d(geo_model, direction='y', show_data=False, show_scalar=True, show_lith=False)

<a id='visualizing-the-computed-model-in-3d'></a>

# Visualizing the computed model in 3D

The computed model can be visualized in 3D using the `pyvista` library. Setting `notebook=False` will open an interactive windows and the model can be rotated and zooming is possible. 

In [ ]:
gpv = gp.plot_3d(geo_model, 
                 image=False, 
                 show_topography=True,
                 plotter_type='basic', 
                 notebook=True, 
                 show_lith=True)

<a id='conclusions'></a>
# Conclusions

<div class="alert alert-block alert-success">
<b>In this tutorial, you have learnt the following:</b> <br>
- Get an understanding of how faults are modeled/displayed in GemPy<br>
- How to build a simple model consisting of <b>faulted layers</b> belonging to <b>one Series</b><br>

</div>


<a id='outlook'></a>
# Outlook

<div class="alert alert-block alert-success">
<b>In the next tutorial, you will learn the following:</b> <br>
- Get an understanding of how unconformities are modeled/displayed in GemPy<br>
- How to build a simple model consisting of <b>unconformable layers</b> belonging to <b>multiple Series</b><br>


</div>

[Take me to the next notebook on Github](https://nbviewer.org/github/cgre-aachen/gemgis_data/blob/main/notebooks/01_basic_modeling/model4_Truncated_Layers.ipynb)

[Take me to the next notebook locally](model4_Truncated_Layers.ipynb)

<img src="../../jose/images/fig1.png" />

<a id='licensing'></a>

## Licensing

Institute for Computational Geoscience, Geothermics and Reservoir Geophysics, RWTH Aachen University & Fraunhofer IEG, Fraunhofer Research Institution for Energy Infrastructures and Geotechnologies IEG, Authors: Alexander Juestel. For more information contact: alexander.juestel(at)ieg.fraunhofer.de

All notebooks are licensed under a Creative Commons Attribution 4.0 International License (CC BY 4.0, http://creativecommons.org/licenses/by/4.0/). References for each displayed map are provided. Most of the maps originate from the books of [Powell (1992)](https://link.springer.com/book/9783540586074) and [Bennison (1990)](https://link.springer.com/book/10.1007/978-1-4615-9630-1). References for maps with unknown origin will gladly be added.